# Refactor the Taxi Fare Prediction Problem with a Pipeline

We will refactor the model you built for the Taxi Fare Prediction Problem using:
- Custom encoders for the distance and time features
- OneHot Encoder in order to encode the hour and day of week features
- SimpleImputer to fill missing values
- A simple linear regression
- A pipeline to put all together

Then: 
- train this pipeline
- apply the pipeline on test data
- generate predictions and submit these new predictions to Kaggle

## First pipeline

In [1]:
import pandas as pd

# import the train dataset ( 1000 first rows)
df = pd.read_csv('train.csv', nrows=1000000)
df.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1


In [2]:
# prepare X and y
X = df.drop('fare_amount', axis=1)
y = df['fare_amount']

In [36]:
from sklearn.model_selection import train_test_split

# Hold out ( train and test split )
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [37]:
X_train.head()

,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
377735,2012-12-16 15:24:39.0000001,2012-12-16 15:24:39 UTC,-73.987194,40.722459,-73.977326,40.751603,1
224462,2011-10-21 13:48:00.00000037,2011-10-21 13:48:00 UTC,-73.968208,40.755407,-73.994767,40.745412,1
108948,2010-08-11 20:51:00.0000002,2010-08-11 20:51:00 UTC,-73.984532,40.746570,-73.985017,40.756163,5
866612,2013-12-02 21:40:32.0000004,2013-12-02 21:40:32 UTC,-73.983990,40.765013,-74.000060,40.761122,1
673476,2010-09-10 15:33:00.000000132,2010-09-10 15:33:00 UTC,-73.973942,40.763312,-73.990487,40.751437,1


### Custom transformers

With the Taxi Fare Prediction Challenge data, using `BaseEstimator` and `TransformerMixin`, implement:

- a transformer that computes the haversine distance between the pickup and dropoff locations
- a custom encoder that extracts the time features from `pickup_datetime`

In [38]:
import numpy as np

def haversine_vectorized(df, 
                         start_lat="pickup_latitude",
                         start_lon="pickup_longitude",
                         end_lat="dropoff_latitude",
                         end_lon="dropoff_longitude"):
    """ 
        Calculates the great circle distance between two points 
        on the earth (specified in decimal degrees).
        Vectorized version of the haversine distance for pandas df.
        Computes the distance in kms.
    """

    lat_1_rad, lon_1_rad = np.radians(df[start_lat].astype(float)), np.radians(df[start_lon].astype(float))
    lat_2_rad, lon_2_rad = np.radians(df[end_lat].astype(float)), np.radians(df[end_lon].astype(float))
    dlon = lon_2_rad - lon_1_rad
    dlat = lat_2_rad - lat_1_rad

    a = np.sin(dlat / 2.0) ** 2 + np.cos(lat_1_rad) * np.cos(lat_2_rad) * np.sin(dlon / 2.0) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    return 6371 * c

In [39]:
# create a DistanceTransformer
from sklearn.base import BaseEstimator, TransformerMixin

class DistanceTransformer(BaseEstimator, TransformerMixin):
    """
        Computes the haversine distance between two GPS points.
        Returns a copy of the DataFrame X with only one column: 'distance'.
    """

    def __init__(self,
                 start_lat="pickup_latitude",
                 start_lon="pickup_longitude",
                 end_lat="dropoff_latitude",
                 end_lon="dropoff_longitude"):
        # A COMPPLETER
        self.start_lat = start_lat
        self.start_lon = start_lon
        self.end_lat = end_lat
        self.end_lon = end_lon
    
    def fit(self, X, y=None):
        # A COMPLETER
        return self

    def transform(self, X, y=None):
        X_ = X.copy()
        X_['distance'] = haversine_vectorized(X_,
                                              self.start_lat,
                                             self.start_lon,
                                             self.end_lat,
                                             self.end_lon)
        return X_[['distance']]

In [40]:
# test the DistanceTransformer

dist_trans = DistanceTransformer()
distance = dist_trans.fit_transform(X_train, y_train)
distance.head()

,distance
377735,3.345618
224462,2.498093
108948,1.067475
866612,1.420902
673476,1.919779


In [63]:
# create a TimeFeaturesEncoder
class TimeFeaturesEncoder(BaseEstimator, TransformerMixin):
    """
        Extracts the day of week (dow), the hour, the month and the year from a time column.
        Returns a copy of the DataFrame X with only four columns: 'dow', 'hour', 'month', 'year'.
    """

    def __init__(self, time_column = 'pickup_datetime'):
       # A COMPLETER
        self.time_column = time_column

    def fit(self, X, y=None):
        # A COMPLETER
        return self

    def transform(self, X, y=None):
        # A COMPLETER 
        X_ = X.copy()
        X_[self.time_column] = pd.to_datetime(X_[self.time_column])
        X_['day_of_week'] = X_[self.time_column].dt.dayofweek
        X_['hour'] = X_[self.time_column].dt.hour
        X_['month'] = X_[self.time_column].dt.month
        X_['year'] = X_[self.time_column].dt.year
        return X_[['day_of_week', 'hour', 'month', 'year']]

In [64]:
# test the TimeFeaturesEncoder

time_enc = TimeFeaturesEncoder(time_column='pickup_datetime')
time_features = time_enc.fit_transform(X_train, y_train)
time_features.head()

,day_of_week,hour,month,year
377735,6,15,12,2012
224462,4,13,10,2011
108948,2,20,8,2010
866612,0,21,12,2013
673476,4,15,9,2010


###  Prepocessing pipeline

In [10]:
# visualizing pipelines in HTML
from sklearn import set_config; set_config(display='diagram')

#### Distance pipeline

Create a pipeline for distances:
- convert the pickup and dropoff coordinates into distances with the DistanceTransformer
- standardize these distances

In [11]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# create distance pipeline dist_pipe
dist_pipe = Pipeline([
    ('distance_transformer', DistanceTransformer()),
    ('scaler', StandardScaler())
])

# display distance pipeline
dist_pipe

Pipeline(steps=[('distance_transformer', DistanceTransformer()),
                ('scaler', StandardScaler())])

#### Time features pipeline

Create a pipeline for time features
- extract time features from pickup datetime with the TimeFeaturesEncoder
- encode these categorical time features with the OneHotEncoder

In [12]:
from sklearn.preprocessing import OneHotEncoder

# create time pipeline time_pipe
time_pipe = Pipeline([
    ('time_encoder', TimeFeaturesEncoder()),
    ('one_hot_encoder', OneHotEncoder(handle_unknown='ignore'))
])

# display time pipeline
time_pipe

Pipeline(steps=[('time_encoder', TimeFeaturesEncoder()),
                ('one_hot_encoder', OneHotEncoder(handle_unknown='ignore'))])

#### Preprocessing pipeline

Wrap up the distance pipeline and the time pipeline into a preprocessing pipeline.

In [52]:
from sklearn.compose import ColumnTransformer

distance_cols = ['pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude']
time_cols = ['pickup_datetime']

# create preprocessing pipeline preproc_pipe
preproc_pipe = ColumnTransformer([
    ('distance_transformer', dist_pipe, distance_cols),
    ('time_transformer', time_pipe, time_cols)
])

# display preprocessing pipeline
preproc_pipe

ColumnTransformer(transformers=[('distance_transformer',
                                 Pipeline(steps=[('distance_transformer',
                                                  DistanceTransformer()),
                                                 ('scaler', StandardScaler())]),
                                 ['pickup_latitude', 'pickup_longitude',
                                  'dropoff_latitude', 'dropoff_longitude']),
                                ('time_transformer',
                                 Pipeline(steps=[('time_encoder',
                                                  TimeFeaturesEncoder()),
                                                 ('one_hot_encoder',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['pickup_datetime'])])

### Model pipeline

Create a pipeline containing the preprocessing and the regression model of your choice.

In [53]:
from sklearn.linear_model import LinearRegression

# Add the model of your choice to the pipeline nammed pipe
reg_model = LinearRegression()
pipe = Pipeline(steps=[
    ("preprocessing", preproc_pipe),
    ('regression', reg_model)
])

# display the pipeline with model
pipe

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('distance_transformer',
                                                  Pipeline(steps=[('distance_transformer',
                                                                   DistanceTransformer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['pickup_latitude',
                                                   'pickup_longitude',
                                                   'dropoff_latitude',
                                                   'dropoff_longitude']),
                                                 ('time_transformer',
                                                  Pipeline(steps=[('time_encoder',
                                                                   TimeFeaturesEncoder()),
                                                                  ('one_hot_encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['pickup_datetime'])])),
                ('regression', LinearRegression())])

In [55]:
from sklearn.impute import SimpleImputer

# modify the preproc_pipe to include SimpleImputer
dist_pipe = Pipeline([
    ('distance_transformer', DistanceTransformer()),
    ('scaler', StandardScaler())
])
time_pipe = Pipeline([
    ('time_encoder', TimeFeaturesEncoder()),
    ('one_hot_encoder', OneHotEncoder(handle_unknown='ignore'))
])
preproc_pipe = Pipeline([
    ('distance_pipeline', dist_pipe),
    ('time_pipeline', time_pipe),
    ('imputer', SimpleImputer(strategy='mean'))  # add SimpleImputer here
])

# create a new pipeline including the imputer
reg_model = LinearRegression()
pipe = Pipeline(steps=[
    ("preprocessing", preproc_pipe),
    ('regression', reg_model)
])

### Training and performance

Train the pipelined model and compute the prediction on the test set:

In [56]:
# train the pipelined model
pipe.fit(X_train, y_train)

# compute y_pred on the test set
y_pred = pipe.predict(X_test)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

Use the RMSE to evaluate the performance of the model:

In [44]:
from sklearn.metrics import mean_squared_error

def compute_rmse(y_pred, y_true):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    return rmse

In [45]:
# call compute_rmse
compute_rmse(y_pred, y_test)

NameError: name 'y_pred' is not defined

## Complete the workflow with a pipeline

Here we will implement the whole workflow for our Taxifare kaggle challenge.

For that we will refactor the code in functions for more clarity.

Implement the following functions:
- `get_data()` to fetch the data 
- `clean_data()` to clean the data
- `get_pipeline()` to get the pipeline defined earlier
- `train()` to train our model
- `evaluate()` to evaluate our model on test data

In [46]:
# implement get_data() function
def get_data(nrows=10000):
    '''returns a DataFrame with nrows from s3 bucket'''
    # A COMPLETER
    df = pd.read_csv('path/to/file.csv', nrows=nrows)
    return df

In [47]:
# implement clean_data() function
def clean_data(df, test=False):
    '''returns a DataFrame without outliers and missing values'''
    # A COMPLETER
    df = df[(df['distance'] >= 0) & (df['distance'] <= 100)]
    df = df[(df['fare_amount'] >= 2.5) & (df['fare_amount'] <= 100)]
    df.dropna(inplace = True)
    if not test:
        df = df[df['fare_amount'] > 0]
    return df

In [48]:
# implement set_pipeline() function
def set_pipeline():
    '''returns a pipelined model'''
     # A  COMPLETER
    pipe = Pipeline([
        ('encoder', TimeFeaturesEncoder()),
        ('regressor', RandomForestRegressor())
    ])
    return pipe

In [49]:
# implement train() function
def train(X_train, y_train, pipeline):
    '''returns a trained pipelined model'''
   # A COMPLETER
    pipeline.fit(X_train, y_train)
    return pipeline

In [50]:
# implement evaluate() function
def evaluate(X_test, y_test, pipeline):
    '''returns the value of the RMSE'''
    # A COMPLETER
    y_pred = pipeline.predict(X_test)
    rmse = compute_rmse(y_pred, y_test)
    print(rmse)
    return rmse

### Test the complete worflow

Use the above functions to test the complete workflow.

In [51]:
# store the data in a DataFrame
df = get_data()

# set X and y
y = df["fare_amount"]
X = df.drop("fare_amount", axis=1)

# hold out
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15)

# build pipeline
pipeline = set_pipeline()

# train the pipeline
train(X_train, y_train, pipeline)

# evaluate the pipeline
rmse = evaluate(X_val, y_val, pipeline)

FileNotFoundError: [Errno 2] No such file or directory: 'path/to/file.csv'

### Congrats!

Now we are ready to convert this complete workflow into a packaged code 🚀